In [1]:
import csv
import pandas as pd 
import torch
import torch.nn as nn


In [2]:
from datasets import load_dataset

# s='train[:10]'
dataset_train = load_dataset('csv', data_files='/kaggle/input/translated-data/Translated_hindi_train.csv')
# dataset_val = load_dataset('csv', data_files='/kaggle/input/summariser-data2022/val.csv')

Generating train split: 0 examples [00:00, ? examples/s]

In [3]:
dataset_train

DatasetDict({
    train: Dataset({
        features: ['Id', 'Heading', 'Summary', 'Article'],
        num_rows: 21225
    })
})

In [4]:
df_train = pd.read_csv('/kaggle/input/2023-data/hindi_train.csv')     
# df_val = pd.read_csv('/kaggle/input/summariser-data2022/val.csv')     

In [5]:
article_limit = 300
# summary_limit=
def resize(article,summary):
    a=[]
    s=[]
    for i in range(len(article)):
        if(len(article[i].split())>article_limit):
            a.append(article[i])
            s.append(summary[i])
        else:
            continue
    
    return a,s

In [6]:
train_article = df_train['Article'].tolist()
train_summary = df_train['Summary'].tolist()
# train_article,train_summary = resize(train_article,train_summary)

In [7]:
len(train_article)

21225

In [8]:
# from torch.utils.data import Dataset
from datasets import DatasetDict, Dataset

x=None

t=int(len(train_article)*0.9)
print(t)

data_dict = {
    "Article": train_article[:t],
    "Summary": train_summary[:t]
}

# Create a Dataset object from the dictionary
dataset_train = Dataset.from_dict(data_dict)

dict_train = DatasetDict({
    'train': dataset_train,
    # Add more datasets for validation, test, etc., as needed
})

print((dict_train))
# print(t)

19102
DatasetDict({
    train: Dataset({
        features: ['Article', 'Summary'],
        num_rows: 19102
    })
})


In [9]:

# val_article = df_val['Article'].tolist()[:x]
# val_summary = df_val['Summary'].tolist()[:x]

val_article = train_article[t:]
val_summary = train_summary[t:] 

# val_article,val_summary = resize(val_article,val_summary)

data_dict = {
    "Article": val_article,
    "Summary": val_summary
}

# Create a Dataset object from the dictionary
dataset_val = Dataset.from_dict(data_dict)

dict_val = DatasetDict({
    'validation': dataset_val,
    # Add more datasets for validation, test, etc., as needed
})

print(type(dict_val))

<class 'datasets.dataset_dict.DatasetDict'>


In [10]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("ai4bharat/IndicBART")
model = AutoModelForSeq2SeqLM.from_pretrained("ai4bharat/IndicBART")
# from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# tokenizer = AutoTokenizer.from_pretrained("Someman/bart-hindi")
# model = AutoModelForSeq2SeqLM.from_pretrained("Someman/bart-hindi")


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


tokenizer_config.json:   0%|          | 0.00/498 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/832 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.90M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/221 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/398 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


pytorch_model.bin:   0%|          | 0.00/976M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


MBartForConditionalGeneration(
  (model): MBartModel(
    (shared): Embedding(64014, 1024, padding_idx=0)
    (encoder): MBartEncoder(
      (embed_tokens): Embedding(64014, 1024, padding_idx=0)
      (embed_positions): MBartLearnedPositionalEmbedding(1026, 1024)
      (layers): ModuleList(
        (0-5): 6 x MBartEncoderLayer(
          (self_attn): MBartAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
          (final_layer_norm): La

In [11]:
from transformers import AutoTokenizer
import torch

df_test = pd.read_csv('/kaggle/input/summariser-data2022/test.csv')
text= df_test['Article'].tolist()[12]

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

inputs = tokenizer(text, return_tensors="pt")

# Move the input tensors to the same device as the model
inputs = {k: v.to(device) for k, v in inputs.items()}

# Generate outputs using the model
with torch.no_grad():
    # Access the correct key for `input_ids` in the `inputs` dictionary
    input_ids = inputs["input_ids"]
    outputs = model.generate(input_ids, max_new_tokens=70, do_sample=False)

generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print("Generated Text:", generated_text)


Generated Text: देहरादून लखनऊ. उत्तराखंड के चमोली जिला में बड़ी आपदा में उत्तर प्रदेश सरकार ने मदद के लिए अपने दरवाजे खोल दिए हैं। इसके साथ ही वहां पर फंसे लोगों को बाहर निकालने में सहयोग देने के साथ यूपी सरकार ने इमरजेंसी ऑपरेशन कंट्रोल रूम भी स्थापित किया। जिसके लिए हेल्पलाइन के साथ व्हाट्सएप नम्बर भी जारी किया गया है। मुख्यमंत्री योगी ने


In [12]:
summary = df_test['Summary'].tolist()[12]
summary

'सात फरवरी को आयी आपदा में यहां पर लखीमपुर खीरी के साथ ही सहारनपुर, अमरोहा, श्रावस्ती के साथ अन्य जिलों के सैकड़ों लोगों के फंसे होने की संभावना है। इस संबंध में राहत आयुक्त उत्तर प्रदेश लगातार उत्तराखंड सरकार के साथ समन्वय कर रहे हैं। किसी भी प्रकार की जानकारी मिलने पर पीड़ित से सम्पर्क किया जा रहा है।'

In [13]:
text

'लखनऊ. उत्तराखंड के चमोली जिला में बड़ी आपदा में उत्तर प्रदेश सरकार ने मदद के लिए अपने दरवाजे खोल दिए हैं। इसके साथ ही वहां पर फंसे लोगों को बाहर निकालने में सहयोग देने के साथ यूपी सरकार ने इमरजेंसी ऑपरेशन कंट्रोल रूम भी स्थापित किया। जिसके लिए हेल्पलाइन के साथ व्हाट्सएप नम्बर भी जारी किया गया है। मुख्यमंत्री योगी ने उत्तराखंड राज्य सरकार से समन्वय के लिए दो अधिकारियों को देहरादून भेजने के निर्देश दिए हैं। साथ ही गन्ना विकास मंत्री सुरेश राणा से उत्तराखंड सरकार से समन्वय स्थापित कर प्रदेश के प्रभावित और लापता लोगों की खोज की कार्रवाई के आदेश दिए हैं।पढ़ें- TMC सांसद महुआ मोइत्रा ने पूर्व सीजेआई की आलोचना कीसात फरवरी को आयी आपदा में यहां पर लखीमपुर खीरी के साथ ही सहारनपुर, अमरोहा, श्रावस्ती के साथ अन्य जिलों के सैकड़ों लोगों के फंसे होने की संभावना है। इस संबंध में राहत आयुक्त उत्तर प्रदेश लगातार उत्तराखंड सरकार के साथ समन्वय कर रहे हैं। किसी भी प्रकार की जानकारी मिलने पर पीड़ित से सम्पर्क किया जा रहा है। घायल या फिर चोटिल होने की स्थिति में उनके इलाज की व्यवस्था की जा रही है। इसके साथ 

In [14]:
max_input_length = 600
max_target_length = 40

prefix="summarize: "
x=10
def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["Article"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True,padding='max_length')

    # Setup the tokenizer for targets
    labels = tokenizer(text_target=examples["Summary"], max_length=max_target_length, truncation=True,padding='max_length')
#     labels = tokenizer(text_target=examples["Summary"], padding='longest', truncation=True)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [15]:
# inp = preprocess_function(dataset_train['train'])

# x=10
# tokenised_data_train = dataset_train.map(preprocess_function, batched=True)
# tokenised_data_val = dataset_val.map(preprocess_function, batched=True)

tokenised_data_train = dict_train.map(preprocess_function, batched=True)
tokenised_data_val = dict_val.map(preprocess_function, batched=True)

Map:   0%|          | 0/19102 [00:00<?, ? examples/s]

Map:   0%|          | 0/2123 [00:00<?, ? examples/s]

In [16]:
type(tokenised_data_train)

datasets.dataset_dict.DatasetDict

In [17]:
import nltk
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]
    
    # Note that other metrics may not have a `use_aggregator` parameter
    # and thus will return a list, computing a metric for each sentence.
    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True, use_aggregator=True)
    # Extract a few results
    result = {key: value * 100 for key, value in result.items()}
    
    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    
    return {k: round(v, 4) for k, v in result.items()}

In [18]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model='ai4bharat/IndicBART')

2024-04-24 23:04:02.687690: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-24 23:04:02.687825: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-24 23:04:02.819376: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [19]:
!pip install rouge_score
!pip install evaluate

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=59937413a54f320f37f3270b1c18406779975cf749b6d85b6310d19cd75c851a
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 1.5 MB/s eta 0:00:00a 0:00:01


In [20]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
import evaluate

rouge = evaluate.load("rouge")


batch_size = 8
# model_name = model_checkpoint.split("/")[-1]
args = Seq2SeqTrainingArguments(
    output_dir="Indic_Bart",
#     eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=5,
    num_train_epochs=2,
    predict_with_generate=True,
    fp16=True,
)

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [21]:
torch.cuda.empty_cache()

In [22]:
tokenised_data_train

DatasetDict({
    train: Dataset({
        features: ['Article', 'Summary', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 19102
    })
})

In [23]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenised_data_train["train"],
    eval_dataset=tokenised_data_val["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
#      data_collator=data_collator,
)

trainer.train()

/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss
500,2.655100
1000,2.151800
1500,1.935300
2000,1.840500


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'forced_eos_token_id': 2}
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'forced_eos_token_id': 2}
/opt/conda/lib/python3.1

TrainOutput(global_step=2388, training_loss=2.091841731239204, metrics={'train_runtime': 3758.2463, 'train_samples_per_second': 10.165, 'train_steps_per_second': 0.635, 'total_flos': 2.42563325755392e+16, 'train_loss': 2.091841731239204, 'epoch': 2.0})

In [24]:
print()

In [25]:
text = df_train['Article'].tolist()[11]
text='summarise: '+text
text

'summarise: सरकारी नौकरी की तलाश में जुटे युवाओं के लिए हम फिर से 5 लेटेस्ट नौकरियों की जानकारी के साथ हाजिर हैं। राजस्थान हाईकोर्ट में स्टेनोग्राफर के 277 पदों पर वैकेंसी है। आवेदन की शुरुआत एक अगस्त 2023 से होगी।\nरेल इंडिया टेक्निकल एंड इकोनॉमिक सर्विस (RITES) ने भारत के रेल मंत्रालय के तहत ड्राफ्ट्समैन सिविल इंजीनियर, पर्यावरण सामाजिक निगरानी विशेषज्ञ, जूनियर डिजाइन इंजीनियर के 111 पदों के लिए ऑनलाइन आवेदन शुरू कर दिया है। 7 अगस्त तक ऑनलाइन आवेदन कर सकते हैं।\nभुवनेश्वर AIIMS में सीनियर नर्सिंग ऑफिसर, स्टोर कीपर, कैशियर, कनिष्ठ प्रशासनिक सहायक, मेडिकल रिकॉर्ड तकनीशियन (रिकॉर्ड क्लर्क), वायरमैन, फार्मासिस्ट के 775 पदों के लिए वैकेंसी थी। अगर आप अभी तक अप्लाई नहीं कर सके हैं तो आज यानी 30 जुलाई लास्ट डेट है।\nBHU यानी बनारस हिंदू विश्वविद्यालय ने असिस्टेंट प्रोफेसर, एसोसिएट प्रोफेसर और प्रोफेसर के 307 पदों पर भर्ती निकाली है। अप्लाई करने की लास्ट डेट कल यानी 31 जुलाई 2023 है।\nनेत्र परीक्षण अधिकारी के 157 पदों पर भर्ती निकली है। आवेदन की आखिरी डेट 7 अगस्त 2023 है। सिलेक्शन होने पर 25

In [26]:
import pickle
with open('Indic_Bart.pkl', 'wb') as f:
    pickle.dump(model, f)



In [27]:
tokenizer.save_pretrained('tokeniser')

('tokeniser/tokenizer_config.json',
 'tokeniser/special_tokens_map.json',
 'tokeniser/spiece.model',
 'tokeniser/added_tokens.json',
 'tokeniser/tokenizer.json')

In [28]:
import pandas as pd

df_test = pd.read_csv('/kaggle/input/summariser-data2022/test.csv')

In [29]:
from transformers import pipeline
# prin
with open('/kaggle/working/Indic_Bart.pkl', 'rb') as f:
    model = pickle.load(f)
text= df_test['Article'].tolist()[12]
summarizer = pipeline("summarization", model=model,tokenizer=AutoTokenizer.from_pretrained("/kaggle/working/tokeniser"))
summarizer(text)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1132: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


[{'summary_text': 'उत्तर प्रदेश सरकार ने मदद के लिए अपने दरवाजे खोल दिए हैं। इसके साथ ही वहां'}]

In [30]:
text

'लखनऊ. उत्तराखंड के चमोली जिला में बड़ी आपदा में उत्तर प्रदेश सरकार ने मदद के लिए अपने दरवाजे खोल दिए हैं। इसके साथ ही वहां पर फंसे लोगों को बाहर निकालने में सहयोग देने के साथ यूपी सरकार ने इमरजेंसी ऑपरेशन कंट्रोल रूम भी स्थापित किया। जिसके लिए हेल्पलाइन के साथ व्हाट्सएप नम्बर भी जारी किया गया है। मुख्यमंत्री योगी ने उत्तराखंड राज्य सरकार से समन्वय के लिए दो अधिकारियों को देहरादून भेजने के निर्देश दिए हैं। साथ ही गन्ना विकास मंत्री सुरेश राणा से उत्तराखंड सरकार से समन्वय स्थापित कर प्रदेश के प्रभावित और लापता लोगों की खोज की कार्रवाई के आदेश दिए हैं।पढ़ें- TMC सांसद महुआ मोइत्रा ने पूर्व सीजेआई की आलोचना कीसात फरवरी को आयी आपदा में यहां पर लखीमपुर खीरी के साथ ही सहारनपुर, अमरोहा, श्रावस्ती के साथ अन्य जिलों के सैकड़ों लोगों के फंसे होने की संभावना है। इस संबंध में राहत आयुक्त उत्तर प्रदेश लगातार उत्तराखंड सरकार के साथ समन्वय कर रहे हैं। किसी भी प्रकार की जानकारी मिलने पर पीड़ित से सम्पर्क किया जा रहा है। घायल या फिर चोटिल होने की स्थिति में उनके इलाज की व्यवस्था की जा रही है। इसके साथ 

In [31]:
t=df_test['Summary'].tolist()[12]
t

'सात फरवरी को आयी आपदा में यहां पर लखीमपुर खीरी के साथ ही सहारनपुर, अमरोहा, श्रावस्ती के साथ अन्य जिलों के सैकड़ों लोगों के फंसे होने की संभावना है। इस संबंध में राहत आयुक्त उत्तर प्रदेश लगातार उत्तराखंड सरकार के साथ समन्वय कर रहे हैं। किसी भी प्रकार की जानकारी मिलने पर पीड़ित से सम्पर्क किया जा रहा है।'

In [32]:
with open('/kaggle/working/Indic_Bart.pkl', 'rb') as f:
    model = pickle.load(f)
tokenizer = AutoTokenizer.from_pretrained("/kaggle/working/tokeniser")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [33]:
from transformers import AutoTokenizer
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

inputs = tokenizer(text, return_tensors="pt")

# Move the input tensors to the same device as the model
inputs = {k: v.to(device) for k, v in inputs.items()}

# Generate outputs using the model
with torch.no_grad():
    # Access the correct key for `input_ids` in the `inputs` dictionary
    input_ids = inputs["input_ids"]
    outputs = model.generate(input_ids, max_new_tokens=100, do_sample=False)

generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print("Generated Text:", generated_text)


Generated Text: उत्तराखंड के चमोली जिला में बड़ी आपदा में उत्तर प्रदेश सरकार ने मदद के लिए अपने दरवाजे खोल दिए हैं। इसके साथ ही वहां पर फंसे लोगों को बाहर निकालने में सहयोग देने के साथ यूपी सरकार ने इमरजेंसी ऑपरेशन कंट्रोल रूम भी स्थापित किया। जिसके लिए हेल्पलाइन के साथ व्हाट्सएप नम्बर भी जारी किया गया है। मुख्यमंत्री योगी ने उत्तराखंड राज्य सरकार से समन्वय के लिए दो अधिकारियों को देहरादून भेजने के निर्देश दिए हैं। साथ ही लोगों को उनके घर वापस भेजने का भी इंतजाम किया जा रहा है


In [34]:
text

'लखनऊ. उत्तराखंड के चमोली जिला में बड़ी आपदा में उत्तर प्रदेश सरकार ने मदद के लिए अपने दरवाजे खोल दिए हैं। इसके साथ ही वहां पर फंसे लोगों को बाहर निकालने में सहयोग देने के साथ यूपी सरकार ने इमरजेंसी ऑपरेशन कंट्रोल रूम भी स्थापित किया। जिसके लिए हेल्पलाइन के साथ व्हाट्सएप नम्बर भी जारी किया गया है। मुख्यमंत्री योगी ने उत्तराखंड राज्य सरकार से समन्वय के लिए दो अधिकारियों को देहरादून भेजने के निर्देश दिए हैं। साथ ही गन्ना विकास मंत्री सुरेश राणा से उत्तराखंड सरकार से समन्वय स्थापित कर प्रदेश के प्रभावित और लापता लोगों की खोज की कार्रवाई के आदेश दिए हैं।पढ़ें- TMC सांसद महुआ मोइत्रा ने पूर्व सीजेआई की आलोचना कीसात फरवरी को आयी आपदा में यहां पर लखीमपुर खीरी के साथ ही सहारनपुर, अमरोहा, श्रावस्ती के साथ अन्य जिलों के सैकड़ों लोगों के फंसे होने की संभावना है। इस संबंध में राहत आयुक्त उत्तर प्रदेश लगातार उत्तराखंड सरकार के साथ समन्वय कर रहे हैं। किसी भी प्रकार की जानकारी मिलने पर पीड़ित से सम्पर्क किया जा रहा है। घायल या फिर चोटिल होने की स्थिति में उनके इलाज की व्यवस्था की जा रही है। इसके साथ 

In [35]:
t=df_test['Summary'].tolist()[12]
t

'सात फरवरी को आयी आपदा में यहां पर लखीमपुर खीरी के साथ ही सहारनपुर, अमरोहा, श्रावस्ती के साथ अन्य जिलों के सैकड़ों लोगों के फंसे होने की संभावना है। इस संबंध में राहत आयुक्त उत्तर प्रदेश लगातार उत्तराखंड सरकार के साथ समन्वय कर रहे हैं। किसी भी प्रकार की जानकारी मिलने पर पीड़ित से सम्पर्क किया जा रहा है।'

In [36]:
text

'लखनऊ. उत्तराखंड के चमोली जिला में बड़ी आपदा में उत्तर प्रदेश सरकार ने मदद के लिए अपने दरवाजे खोल दिए हैं। इसके साथ ही वहां पर फंसे लोगों को बाहर निकालने में सहयोग देने के साथ यूपी सरकार ने इमरजेंसी ऑपरेशन कंट्रोल रूम भी स्थापित किया। जिसके लिए हेल्पलाइन के साथ व्हाट्सएप नम्बर भी जारी किया गया है। मुख्यमंत्री योगी ने उत्तराखंड राज्य सरकार से समन्वय के लिए दो अधिकारियों को देहरादून भेजने के निर्देश दिए हैं। साथ ही गन्ना विकास मंत्री सुरेश राणा से उत्तराखंड सरकार से समन्वय स्थापित कर प्रदेश के प्रभावित और लापता लोगों की खोज की कार्रवाई के आदेश दिए हैं।पढ़ें- TMC सांसद महुआ मोइत्रा ने पूर्व सीजेआई की आलोचना कीसात फरवरी को आयी आपदा में यहां पर लखीमपुर खीरी के साथ ही सहारनपुर, अमरोहा, श्रावस्ती के साथ अन्य जिलों के सैकड़ों लोगों के फंसे होने की संभावना है। इस संबंध में राहत आयुक्त उत्तर प्रदेश लगातार उत्तराखंड सरकार के साथ समन्वय कर रहे हैं। किसी भी प्रकार की जानकारी मिलने पर पीड़ित से सम्पर्क किया जा रहा है। घायल या फिर चोटिल होने की स्थिति में उनके इलाज की व्यवस्था की जा रही है। इसके साथ 

In [37]:
!pip install indic-nlp-library


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 16.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.7/92.7 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.0/120.0 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.5/83.5 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.4/89.4 kB 7.1 MB/s eta 0:00:00


In [38]:
from transformers import AutoTokenizer
import torch

# with open('/kaggle/working/Indic_Bart.pkl', 'rb') as f:
#     model = pickle.load(f)
# tokenizer = AutoTokenizer.from_pretrained("/kaggle/working/tokeniser")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# def summarise(s):

inputs = tokenizer(text, return_tensors="pt")

# Move the input tensors to the same device as the model
inputs = {k: v.to(device) for k, v in inputs.items()}

# Generate outputs using the model
with torch.no_grad():
    # Access the correct key for `input_ids` in the `inputs` dictionary
    input_ids = inputs["input_ids"]
    outputs = model.generate(input_ids, max_new_tokens=80, do_sample=False)

generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
# return generated_text
print("Generated Text:", generated_text)

Generated Text: उत्तराखंड के चमोली जिला में बड़ी आपदा में उत्तर प्रदेश सरकार ने मदद के लिए अपने दरवाजे खोल दिए हैं। इसके साथ ही वहां पर फंसे लोगों को बाहर निकालने में सहयोग देने के साथ ही सहारनपुर, अमरोहा, श्रावस्ती के साथ अन्य जिलों के सैकड़ों लोगों के फंसे होने की संभावना है। इस संबंध में राहत आयुक्त उत्तर प्रदेश लगातार उत्तराखंड सरकार के साथ समन्वय कर रहे हैं।


In [39]:
summaries = []
ori_summary = []
s=df_test['Summary'].tolist()
a = df_test['Article'].tolist()
print(len(s))
ori_summary.append(s[0])
print(a[0])
# summaries.append(summarise(a[0]))
# for i in range(len(s)):
#     ori_summary.append(s[i])
#     summaries.append(summarise(a[i]))

2842
नई दिल्ली. भारतीय जनता पार्टी ने अगले साल होने वाले चुनावों के चुनाव प्रभारियों की नियुक्तियां की हैं। भाजपा के लिहाज से सबसे महत्वपूर्ण राज्य उत्तर प्रदेश का जिम्मा धर्मेंद्र प्रधान को दिया है। धर्मेंद्र प्रधान यूपी के चुनाव प्रभारी होंगे। गजेंद्र सिंह शेखावत को पंजाब का प्रभारी बनाया है।उत्तर प्रदेश में भाजपा ने चुनाव के लिए अनुराग ठाकुर, अर्जुन मेघवाल, सरोज पांडेय, शोभा करंदलाजे, कैप्टन अभिमन्यु, अन्नपूर्णा देवी, विवेक ठाकुर को सह प्रभारी की जिम्मेदारी दी है। भाजपा ने उत्तर प्रदेश चुनाव के लिए राज्य में कई संगठन प्रभारी नियुक्त किए हैं। संजय भाटिया को वेस्टर्न यूपी, वाय. सत्या ठाकुर को अवध, संजीव चौरसिया को बृज, सुधीर गुप्ता को कानपुर, सुनील ओझा को काशी का संगठन प्रभारी नियुक्त किया गया है।केंद्रीय जल शक्ति मंत्री गजेंद्र सिंह शेखावत को पंजाब का प्रभारी बनाया गया है। पंजाब में हरदीप सिंह पुरी, मीनाक्षी लेखी और विनोद चावड़ा सह-प्रभारी नियुक्त किए गए हैं। हिंदी भाषी राज्य उत्तराखंड के लिए प्रल्हाद जोशी को प्रभारी बनाया गया है। प्रल्हाद जोशी केंद्र सरकार में कोयला और खान मंत्री है

In [40]:
!pip install rouge

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [41]:
df_train=df_test[:100]

In [42]:

def predict_summary(text):
    inputs = tokenizer(text, return_tensors='pt', max_length=600, truncation=True)
    inputs.to("cuda")
    summary_ids = model.generate(inputs['input_ids'], num_beams=4, max_length=40, early_stopping=True)
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

df_train['predicted_summary'] = df_train['Article'].apply(predict_summary)

/tmp/ipykernel_34/1709492961.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train['predicted_summary'] = df_train['Article'].apply(predict_summary)


In [ ]:
import pickle
with open('Indic_Bart.pkl', 'wb') as f:
    pickle.dump(model, f)


In [48]:
with open('tokenizer.pkl', 'wb') as f:
    pickle.dump(tokenizer,f)

In [43]:
from rouge import Rouge

rouge = Rouge()


def calculate_rouge(row):
    return rouge.get_scores(row['predicted_summary'], row['Summary'], avg=True)

df_train['rouge'] = df_train.apply(calculate_rouge, axis=1)
df_train['rouge-1'] = df_train['rouge'].apply(lambda x: x['rouge-1']['f'])
df_train['rouge-2'] = df_train['rouge'].apply(lambda x: x['rouge-2']['f'])

print(df_train['rouge-1'].mean())
print(df_train['rouge-2'].mean())

/tmp/ipykernel_34/113450990.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train['rouge'] = df_train.apply(calculate_rouge, axis=1)


0.5612480810776703
0.45123890217177354


/tmp/ipykernel_34/113450990.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train['rouge-1'] = df_train['rouge'].apply(lambda x: x['rouge-1']['f'])
/tmp/ipykernel_34/113450990.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train['rouge-2'] = df_train['rouge'].apply(lambda x: x['rouge-2']['f'])


In [44]:
# # Define compute_metrics function
# def compute_metrics(eval_pred):
#     predictions, labels = eval_pred
#     decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
#     labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
#     decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

#     # Load ROUGE and BERT score metrics
#     rouge = load_metric('rouge')
#     bert_score = load_metric('bertscore')

#     # Compute ROUGE and BERT scores
#     rouge_output = rouge.compute(predictions=decoded_preds, references=decoded_labels, rouge_types=["rougeL", "rouge2"])
#     bert_score_output = bert_score.compute(predictions=decoded_preds, references=decoded_labels, lang="hi")

#     return {
#         "rouge2": rouge_output['rouge2'].mid.fmeasure,
#         "rougeL": rouge_output['rougeL'].mid.fmeasure,
#         "bert_score": bert_score_output['f1'][0]
#     }